# SITCOM-771
Craig Lage - 12-May-23 \

Here is what was requested:
* How did the force profile change over time?
* Find all bump test patterns in the EFD for a given FA.
* Select nominal force patterns and average some of them.
* Overplot Nomial force pattern to actual force pattern.
* Do the subtraction.
* Plot the residuals.

This notebook does those things


## Prepare the notebook

In [ ]:
# Directory to store the data
data_dir = "/home/c/cslage/u/MTM1M3/data/"

# Times of single bump test
single_start = "2023-04-17T10:00:00"
single_end = "2023-04-17T11:15:00"

# Times of multiple bump tests
multiple_start = "2020-06-01T00:00:00"
multiple_end = "2023-04-20T00:00:00"

# Times of test with bump test residuals
residual_start = "2023-04-28T18:10:00"
residual_end = "2023-04-28T19:18:00"

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from pathlib import Path
import pickle as pkl
from astropy.time import Time, TimeDelta
from scipy.interpolate import UnivariateSpline

from lsst.ts.xml.tables.m1m3 import FATable, force_actuator_from_id
from lsst_efd_client import EfdClient

data_path = Path(data_dir)
os.makedirs(data_path, exist_ok=True)

In [ ]:
client = EfdClient('usdf_efd')


## Given an actuator ID, this plots the bump test result for a single test

In [ ]:
async def plot_bump_test_results(fig, bumps, fa):
    """ Plot a visualization of the bump test result for a single test
        Parameters
        ----------
        fig : matplotlib.pyplot.Figure
            a matplotlib figure object
        bumps: pandas dataframe
            This is a dataframe containing the bump test status
        fa: 'ForceActuatorData'
            The desired actuator record from FATable

        Returns
        -------
        No return, only the fig object which was input
    """    
    this_bump = bumps[bumps['actuatorId']==fa.actuator_id]
    # The pass/fail results are actually in the next test.
    last_this_bump_index = bumps[bumps['actuatorId']==id].last_valid_index()
    pass_fail = bumps.iloc[bumps.index.get_loc(last_this_bump_index)+1]
    primary_bump = f"primaryTest{fa.index}"
    primary_force = f"zForce{fa.z_index}"
    if fa.s_index is not None:
        secondary_bump = f"secondaryTest{fa.s_index}"
        secondary_name = fa.orientation.name
        if fa.y_index is not None:
            secondary_force = f"yForce{fa.y_index}"
        else:
            secondary_force = f"xForce{fa.x_index}"
    else:
        secondary_name = None

    plt.subplots_adjust(wspace=0.3)
    plt.subplot(1,2,1)
    plot_start = this_bump[this_bump[primary_bump]==2]['timestamp'].values[0] - 1.0
    plot_end = plot_start + 14.0 #this_bump[this_bump[primary_bump]==5]['timestamp'].values[0] + 2.0
    start = Time(plot_start, format='unix_tai', scale='tai')
    end = Time(plot_end, format='unix_tai', scale='tai')
    forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", \
                                             [primary_force, 'timestamp'], start.utc, end.utc)
    times = forces['timestamp'].values
    t0 = times[0]
    times -= t0
    plot_start -= t0
    plot_end -= t0
    plt.title(f"Primary - Z - ID:{id}")
    plt.plot(times, forces[primary_force].values)
    if pass_fail[primary_bump] == 6:
        plt.text(2.0, 350.0, "PASSED", color='g')
    elif pass_fail[primary_bump] == 7:
        plt.text(2.0, 350.0, "FAILED", color='r')

    plt.xlim(plot_start, plot_end)
    plt.ylim(-400,400)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Force (nt)")
    plt.subplot(1,2,2)
    if secondary_name is not None:
        plt.title(f"Secondary - {secondary_name} - ID:{id}")
        plot_start = this_bump[this_bump[secondary_bump]==2]['timestamp'].values[0] - 1.0
        plot_end = plot_start + 14.0 #this_bump[this_bump[secondary_bump]==5]['timestamp'].values[0] + 2.0
        start = Time(plot_start, format='unix_tai', scale='tai')
        end = Time(plot_end, format='unix_tai', scale='tai')
        forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", \
                                                 [secondary_force, 'timestamp'], start.utc, end.utc)
        times = forces['timestamp'].values
        t0 = times[0]
        times -= t0
        plot_start -= t0
        plot_end -= t0
        plt.plot(times, forces[secondary_force].values)
        if pass_fail[secondary_bump] == 6:
            plt.text(2.0, 350.0, "PASSED", color='g')
        elif pass_fail[secondary_bump] == 7:
            plt.text(2.0, 350.0, "FAILED", color='r')
        plt.xlim(plot_start, plot_end)
        plt.ylim(-400,400)
        plt.xlabel("Time (seconds)")
        plt.ylabel("Force (nt)")
    else:
        plt.title("No Secondary")
        plt.xticks([])
        plt.yticks([])
    return

## Now make the plot for a single actuator

In [ ]:
bumps = await client.select_time_series("lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus", "*", \
                                        Time(single_start, scale='utc'), Time(single_end, scale='utc'))
timestamp = bumps.index[0].isoformat().split('.')[0].replace('-','').replace(':','')
fig = plt.figure(figsize=(10,5))
id = 227
await plot_bump_test_results(fig, bumps, force_actuator_from_id(id))
timestamp = bumps.index[0].isoformat().split('.')[0].replace('-','').replace(':','')
plt.savefig(data_path / f"Bump_Test_{id}_{timestamp}.png")

## This will make a plot of all of the actuators in a given bump test

In [ ]:
bumps = await client.select_time_series("lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus", "*", \
                                        Time(single_start, scale='utc'), Time(single_end, scale='utc'))
timestamp = bumps.index[0].isoformat().split('.')[0].replace('-','').replace(':','')
pdf = PdfPages(data_path / f"Bump_Tests_{timestamp}.pdf")

for fa in FATable:
    fig = plt.figure(figsize=(10,5))
    await plot_bump_test_results(fig, bumps, fa)
    pdf.savefig(fig)  # saves the current figure into a pdf page
    plt.close()
pdf.close()

## Now let's look at the history of many bump tests

In [ ]:
async def plot_multiple_bump_test_results(fig, many_bumps, fa):
    """ Plot a visualization of the bump test results for 
        a dataframe containing many bump tests
        Parameters
        ----------
        fig : matplotlib.pyplot.Figure
             a matplotlib figure object
        many_bumps: pandas dataframe
            This is a dataframe containing the bump test status
        fa: 'ForceActuatorData'
            The desired actuator record from FATable

        Returns
        -------
        No return, only the fig object which was input
    """    
    
    these_bumps = many_bumps[many_bumps['actuatorId']==fa.actuator_id]
    primary_bump = f"primaryTest{fa.index}"
    primary_force = f"zForce{fa.z_index}"
    if fa.s_index is not None:
        secondary_bump = f"secondaryTest{fa.s_index}"
        secondary_name = fa.orientation.name
        if fa.y_index is not None:
            secondary_force = f"yForce{fa.y_index}"
        else:
            secondary_force = f"xForce{fa.x_index}"
    else:
        secondary_name = None
    plt.subplots_adjust(wspace=0.3)
    plt.suptitle(f"Multiple bump tests Actuator ID {fa.actuator_id}", fontsize=18)

    # Now find the separate tests
    times = these_bumps['timestamp'].values
    start_times = []
    end_times = []
    for i, time in enumerate(times):
        if i == 0:
            start_times.append(time)
            continue
        if (time - times[i-1]) > 60.0:
            start_times.append(time)
            end_times.append(times[i-1])
    end_times.append(times[-1])
    num_plots = 0
    pass_count = 0
    fail_count = 0
    for i in range(len(start_times)):
        start_time = start_times[i]
        end_time = end_times[i]
        this_bump = these_bumps[(these_bumps['timestamp'] >= start_time) & (these_bumps['timestamp'] <= end_time)]
        try:
            num_plots += 1
            plt.subplot(1,2,1)
            plot_start = this_bump[this_bump[primary_bump]==2]['timestamp'].values[0] - 1.0
            plot_end = plot_start + 14.0
            start = Time(plot_start, format='unix_tai', scale='tai')
            end = Time(plot_end, format='unix_tai', scale='tai')
            forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [primary_force, 'timestamp'], \
                                                     start.utc, end.utc)
            times = forces['timestamp'].values
            t0 = times[0]
            times -= t0
            plot_start -= t0
            plot_end -= t0
            last_index = bumps[bumps['actuatorId']==fa.actuator_id].last_valid_index()
            pass_fail = bumps.iloc[bumps.index.get_loc(last_index)+1][primary_bump]
            #print(i, pass_fail)
            if pass_fail == 6:
                pass_count += 1
                #print("PASSED")
            elif pass_fail == 7:
                fail_count += 1
                #print("FAILED")

            plt.title("Primary   Z")
            plt.plot(times, forces[primary_force].values)
            plt.xlim(plot_start, plot_end)
            plt.ylim(-400,400)
            plt.xlabel("Time (seconds)")
            plt.ylabel("Force (nt)")
            plt.subplot(1,2,2)
            if secondary_name is not None:
                plt.title(f"Secondary   {secondary_name}")
                plot_start = this_bump[this_bump[secondary_bump]==2]['timestamp'].values[0] - 1.0
                plot_end = plot_start + 14.0
                start = Time(plot_start, format='unix_tai', scale='tai')
                end = Time(plot_end, format='unix_tai', scale='tai')
                forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [secondary_force, 'timestamp'], \
                                                         start.utc, end.utc)
                times = forces['timestamp'].values
                t0 = times[0]
                times -= t0
                plot_start -= t0
                plot_end -= t0
                plt.plot(times, forces[secondary_force].values)
                plt.xlim(plot_start, plot_end)
                plt.ylim(-400,400)
                plt.xlabel("Time (seconds)")
                plt.ylabel("Force (nt)")
            else:
                plt.title("No Secondary")
                plt.xticks([])
                plt.yticks([])
        except:
            continue
            
    plt.subplot(1,2,1)
    plt.text(2.0, 350, f"{num_plots} tests, {pass_count} passed, {fail_count} failed")
    plt.subplot(1,2,2)
    plt.text(2.0, 350, f"{num_plots} tests, {pass_count} passed, {fail_count} failed")
    return

## Now make the history plot for a single actuator

In [ ]:
many_bumps = await client.select_time_series("lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus", "*", \
                                            Time(multiple_start, scale='utc'), Time(multiple_end, scale='utc'))
id = 227
fig = plt.figure(figsize=(10,5))
await plot_multiple_bump_test_results(fig, many_bumps, force_actuator_from_id(id))
plt.savefig(data_path / f"Bump_Test_{id}_Multiple.png")

## This will make a plot of the histories of all of the actuators

In [ ]:
many_bumps = await client.select_time_series("lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus", "*", \
                                            Time(multiple_start, scale='utc'), Time(multiple_end, scale='utc'))
pdf = PdfPages(data_path / f"Bump_Histories.pdf")

for fa in FATable:
    fig = plt.figure(figsize=(10,5))
    await plot_bump_test_results(fig, bumps, fa)
    pdf.savefig(fig)  # saves the current figure into a pdf page
    plt.close()
pdf.close()

## Now generate the dictionary of average splines
## This will be used to plot the residuals
## Note that this only needs to be done once.  After it is run, the average splines are pickled and can be retrieved for future tests.

In [ ]:
async def generate_average_bump_test(many_bumps, average_spline_dict, fa):
    """ generate a spline average of the history of many bump tests
        Parameters
        ----------
        many_bumps: pandas dataframe
            This is a dataframe containing the bump test status
        average_spline_dict: This is a dictionary of the average
            splines.  It is used to get a representative spline
            so we can weed out bad test results from the average
        fa: 'ForceActuatorData'
            The desired actuator record from FATable

        Returns
        -------
        average_primary_spline: A spline representation of the average
            of many bump tests
            
        average_secondary_spline: A spline representation of the average
            of many bump tests
    """    
    
    # Get representative splines to be used to weed out bad runs
    [average_primary_spline, average_secondary_spline] = average_spline_dict[999]
    these_bumps = many_bumps[many_bumps['actuatorId']==fa.actuator_id]
    primary_bump = f"primaryTest{fa.index}"
    primary_force = f"zForce{fa.z_index}"
    if fa.s_index is not None:
        secondary_bump = f"secondaryTest{fa.s_index}"
        secondary_name = fa.orientation.name
        if fa.y_index is not None:
            secondary_force = f"yForce{fa.y_index}"
        else:
            secondary_force = f"xForce{fa.x_index}"
    else:
        secondary_name = None

    # Now find the separate tests
    times = these_bumps['timestamp'].values
    start_times = []
    end_times = []
    for i, time in enumerate(times):
        if i == 0:
            start_times.append(time)
            continue
        if (time - times[i-1]) > 60.0:
            start_times.append(time)
            end_times.append(times[i-1])
    end_times.append(times[-1])
    num_plots = 0
    primary_splines = []
    secondary_splines = []
    for i in range(len(start_times)):
        start_time = start_times[i]
        end_time = end_times[i]
        this_bump = these_bumps[(these_bumps['timestamp'] >= start_time) & (these_bumps['timestamp'] <= end_time)]
        # The pass/fail results are actually in the next test.
        last_this_bump_index = bumps[bumps['actuatorId']==fa.actuator_id].last_valid_index()
        pass_fail = bumps.iloc[bumps.index.get_loc(last_this_bump_index)+1]
        if pass_fail[primary_bump] == 7:
            # Don't include fails
            continue
        try:
            plot_start = this_bump[this_bump[primary_bump]==2]['timestamp'].values[0] - 1.0
            plot_end = plot_start + 14.0
            start = Time(plot_start, format='unix_tai', scale='tai')
            end = Time(plot_end, format='unix_tai', scale='tai')
            forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", \
                                                     [primary_force, 'timestamp'], start.utc, end.utc)
            times = forces['timestamp'].values
            t0 = times[0]
            times -= t0
            primary_forces = forces[primary_force].values
            if average_primary_spline is not None:
                rms_error = np.sqrt(np.mean((primary_forces-average_primary_spline(times))**2))
            else:
                rms_error = 0.0
            if rms_error < 50.0:
                # Only include good fits in the average
                primary_spline = UnivariateSpline(times, primary_forces, s=0.0)
                primary_splines.append(primary_spline)
            if secondary_name is not None:
                if pass_fail[secondary_bump] == 7:
                    # Don't include fails
                    continue
                plot_start = this_bump[this_bump[secondary_bump]==2]['timestamp'].values[0] - 1.0
                plot_end = plot_start + 14.0
                start = Time(plot_start, format='unix_tai', scale='tai')
                end = Time(plot_end, format='unix_tai', scale='tai')
                forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", \
                                                         [secondary_force, 'timestamp'], start.utc, end.utc)
                times = forces['timestamp'].values
                t0 = times[0]
                times -= t0
                secondary_forces = forces[secondary_force].values
                if average_secondary_spline is not None:
                    rms_error = np.sqrt(np.mean((secondary_forces-average_secondary_spline(times))**2))
                else:
                    rms_error = 0.0
                if rms_error < 50.0:
                    # Only include good fits in the average
                    secondary_spline = UnivariateSpline(times, secondary_forces, s=0.0)
                    secondary_splines.append(secondary_spline)
        except:
            continue
    # Now calculate the average spline
    ts = np.linspace(0,14,5000)
    fs = np.zeros_like(ts)
    num_splines = 0
    for spline in primary_splines:
        num_splines += 1
        fs += spline(ts)
    fs /= num_splines
    average_primary_spline = UnivariateSpline(ts, fs)
    fs = np.zeros_like(ts)
    if secondary_name is not None:
        num_splines = 0
        for spline in secondary_splines:
            num_splines += 1
            fs += spline(ts)
        fs /= num_splines
        average_secondary_spline = UnivariateSpline(ts, fs)
    else:
        average_secondary_spline = None
    return [average_primary_spline, average_secondary_spline]

## This cell generates the dictionary with a set of plots of the average spline
### This should only need to be run once

In [ ]:
# First seed the average_spline_dict with a typical bump test
# This is used to weed out bad tests
average_spline_dict = {}
average_spline_dict[999] = [None, None]
[average_primary_spline, average_secondary_spline] = \
await generate_average_bump_test(many_bumps, average_spline_dict, force_actuator_from_id(227))
average_spline_dict[999] = [average_primary_spline, average_secondary_spline]
# Now run all of the actuators
pdf = PdfPages(data_path / "Average_Spline_Dict.pdf")
for fa in FATable:
    primaryName = 'Z'
    secondary_name = fa.orientation.name

    [average_primary_spline, average_secondary_spline] = \
    await generate_average_bump_test(many_bumps, average_spline_dict, fa)
    average_spline_dict[fa.actuator_id] = [average_primary_spline, average_secondary_spline]
    fig = plt.figure(figsize=(10,5))
    ts = np.linspace(0,14,5000)
    plt.suptitle(f"Average Spline bumps for ID = {fa.actuator_id}")
    plt.subplot(1,2,1)
    plt.title(f"Primary  {primaryName}")
    plt.plot(ts, average_primary_spline(ts))
    plt.subplot(1,2,2)
    plt.title(f"Secondary   {secondary_name}")
    if average_secondary_spline is not None:
        plt.plot(ts, average_secondary_spline(ts))
    else:
        plt.xticks([])
        plt.yticks([])

    pdf.savefig(fig)  # saves the current figure into a pdf page
    plt.clf()
pdf.close()


## Pickle the dictionary for future use

In [ ]:
filename = data_path / 'average_spline_dict.pkl'
file = open(filename, 'wb')
pkl.dump(average_spline_dict, file)
file.close()

In [ ]:
filename = data_path / 'average_spline_dict.pkl'
file = open(filename, 'rb')
average_spline_dict = pkl.load(file)
file.close()

## Now plot the residuals against the average

In [ ]:
async def plot_bump_results_and_residuals(fig, bumps, average_spline_dict, fa):
    """ Generate a visualization comparing the current bump test
        to the history, with residuals.
        Parameters
        ----------
        fig : matplotlib.pyplot.Figure
            a matplotlib figure object
        bumps: pandas dataframe
            This is a dataframe containing the bump test status
        average_spline_dict: This is a dictionary containing the average splines.
            The key is the actu
        fa: 'ForceActuatorData'
            The desired actuator record from FATable

        Returns
        -------
        No return, only the fig object which was input
    """    
    
    [average_primary_spline, average_secondary_spline] = average_spline_dict[id]
    this_bump = bumps[bumps['actuatorId']==id]
    index = M1M3FATable.actuatorIDToIndex(id)
    # The pass/fail results are actually in the next test.
    last_this_bump_index = bumps[bumps['actuatorId']==fa.actuator_id].last_valid_index()
    pass_fail = bumps.iloc[bumps.index.get_loc(last_this_bump_index)+1]
    primary_bump = f"primaryTest{fa.index}"
    primary_force = f"zForce{fa.z_index}"
    if fa.s_index is not None:
        secondary_bump = f"secondaryTest{fa.s_index}"
        secondary_name = fa.orientation.name
        if fa.y_index is not None:
            secondary_force = f"yForce{fa.y_index}"
        else:
            secondary_force = f"xForce{fa.x_index}"
    else:
        secondary_name = None

    plt.subplots_adjust(wspace=0.3)
    plt.suptitle(f"Bump Test with Residuals. Actuator ID {fa.actuator_id}", fontsize=18)
    plot_start = this_bump[this_bump[primary_bump]==2]['timestamp'].values[0] - 1.0
    plot_end = plot_start + 14.0 
    start = Time(plot_start, format='unix_tai', scale='tai')
    end = Time(plot_end, format='unix_tai', scale='tai')
    forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", \
                                             [primary_force, 'timestamp'], start.utc, end.utc)
    times = forces['timestamp'].values
    t0 = times[0]
    times -= t0
    primary_forces = forces[primary_force].values
    residuals = primary_forces-average_primary_spline(times)
    rms_error = np.sqrt(np.mean(residuals**2))
    plot_start -= t0
    plot_end -= t0
    plt.subplot(2,2,1)
    plt.title("Primary - Z")
    plt.plot(times, average_primary_spline(times), label='Average')
    plt.plot(times, primary_forces, label='Data')
    if pass_fail[primary_bump] == 6:
        plt.text(2.0, 350.0, "PASSED", color='g')
    elif pass_fail[primary_bump] == 7:
        plt.text(2.0, 350.0, "FAILED", color='r')
    plt.xlim(plot_start, plot_end)
    plt.ylim(-400,400)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Force (nt)")
    plt.legend()
    plt.subplot(2,2,3)
    plt.plot(times, residuals)
    if pass_fail[primary_bump] == 6:
        plt.text(2.0, 75.0, f"RMS = {rms_error:.2f}", color='g')
    elif pass_fail[primary_bump] == 7:
        plt.text(2.0, 75.0, f"RMS = {rms_error:.2f}", color='r')
    plt.xlim(plot_start, plot_end)
    plt.ylim(-100,100)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Residuals (nt)")
    
    if secondary_name is not None:
        plot_start = this_bump[this_bump[secondary_bump]==2]['timestamp'].values[0] - 1.0
        plot_end = plot_start + 14.0
        start = Time(plot_start, format='unix_tai', scale='tai')
        end = Time(plot_end, format='unix_tai', scale='tai')
        forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", \
                                                 [secondary_force, 'timestamp'], start.utc, end.utc)
        times = forces['timestamp'].values
        t0 = times[0]
        times -= t0
        secondary_forces = forces[secondary_force].values
        residuals = secondary_forces-average_secondary_spline(times)
        rms_error = np.sqrt(np.mean(residuals**2))
        plot_start -= t0
        plot_end -= t0
        plt.subplot(2,2,2)
        plt.title(f"Secondary - {secondary_name}")
        plt.plot(times, average_secondary_spline(times), label='Average')
        plt.plot(times, secondary_forces, label='Data')
        if pass_fail[primary_bump] == 6:
            plt.text(2.0, 350.0, "PASSED", color='g')
        elif pass_fail[primary_bump] == 7:
            plt.text(2.0, 350.0, "FAILED", color='r')
        plt.xlim(plot_start, plot_end)
        plt.ylim(-400,400)
        plt.xlabel("Time (seconds)")
        plt.ylabel("Force (nt)")
        plt.legend()
        plt.subplot(2,2,4)
        plt.plot(times, residuals)
        if pass_fail[primary_bump] == 6:
            plt.text(2.0, 75.0, f"RMS = {rms_error:.2f}", color='g')
        elif pass_fail[primary_bump] == 7:
            plt.text(2.0, 75.0, f"RMS = {rms_error:.2f}", color='r')
        plt.xlim(plot_start, plot_end)
        plt.ylim(-100,100)
        plt.xlabel("Time (seconds)")
        plt.ylabel("Residuals (nt)")
    else:
        plt.subplot(2,2,2)
        plt.title("No Secondary")
        plt.xticks([])
        plt.yticks([])
        plt.subplot(2,2,4)
        plt.xticks([])
        plt.yticks([])
    return

## This cell will make the residual plots for a single actuator

In [ ]:
bumps = await client.select_time_series("lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus", "*", \
                                        Time(residual_start, scale='utc'), Time(residual_end, scale='utc'))
timestamp = bumps.index[0].isoformat().split('.')[0].replace('-','').replace(':','')
id = 324
fig = plt.figure(figsize=(10,10))
await plot_bump_results_and_residuals(fig, bumps, average_spline_dict, force_actuator_from_id(id))
plt.savefig(data_path / f"Bump_Test_Residuals_{id}_{timestamp}.png")

## This will make plots of the residuals for the whole bump test

In [ ]:
bumps = await client.select_time_series("lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus", "*", \
                                        Time(residual_start, scale='utc'), Time(residual_end, scale='utc'))
timestamp = bumps.index[0].isoformat().split('.')[0].replace('-','').replace(':','')
pdf = PdfPages(f"/home/c/cslage/u/MTM1M3/data/Bump_Test_Residuals_{timestamp}.pdf")

for fa in FATable:
    fig = plt.figure(figsize=(10,10))
    await plot_bump_results_and_residuals(fig, bumps, average_spline_dict, fa)
    pdf.savefig(fig)  # saves the current figure into a pdf page
    plt.close()
pdf.close()
